# Sending Traces using Python

## Import OpenTelemetry Modules for Traces

In [ ]:
from opentelemetry import baggage, trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import (
    ConsoleSpanExporter,
    BatchSpanProcessor
)
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.trace import Status, StatusCode
import datetime, random, socket, time, uuid

## Create a **TracerProvider** that we can send trace spans to

Create a [TraceProvider](https://github.com/open-telemetry/opentelemetry-specification/blob/main/specification/trace/api.md#tracerprovider) with some **Resource**-level [Semantic Attributes](https://github.com/open-telemetry/semantic-conventions/blob/main/docs/resource/README.md) that describe our service:

In [ ]:
provider = TracerProvider(resource=Resource.create({
    "service.name": __name__,
    "service.instance.id": str(uuid.uuid4()),
    "deployment.environment": "otel",
    "host.name": socket.gethostname(),
    "datadog.host.use_as_metadata": True,
}))
print(provider._resource.__dict__)

## Get a **Tracer** from the **TracerProvider**

Next we create a [BatchSpanProcessor](https://opentelemetry-python.readthedocs.io/en/latest/sdk/trace.export.html#opentelemetry.sdk.trace.export.BatchSpanProcessor) that uses an [OTLPSpanExporter](https://opentelemetry-python.readthedocs.io/en/latest/exporter/otlp/otlp.html#opentelemetry.exporter.otlp.proto.grpc.trace_exporter.OTLPSpanExporter) that is used to send the span to our OTEL Collector:

In [ ]:
provider.add_span_processor(BatchSpanProcessor(OTLPSpanExporter(endpoint="localhost:4317", insecure=True)))
tracer = trace.get_tracer("python", tracer_provider=provider)

We could optionally add a [ConsoleSpanExporter](https://opentelemetry-python.readthedocs.io/en/latest/sdk/trace.export.html#opentelemetry.sdk.trace.export.ConsoleSpanExporter) to the provider that would output all spans from the collector to the console:

```python
provider.add_span_processor(BatchSpanProcessor(ConsoleSpanExporter()))
```

but we will omit this for this lab.

## Create a new trace span and send it to the collector

With the **Tracer**, we can finally start emitting spans. This simple example create a parent span and a child span simulating one microservice invoking another service:

In [ ]:
def frontend():
    with tracer.start_as_current_span("frontend") as frontend_span:
        print("Processing web transaction...")
        time.sleep(random.random())
        handle_checkout()
        time.sleep(random.random())
    
        frontend_span.set_status(Status(StatusCode.OK))
        print("Transaction complete.")
        trace_id = trace.format_trace_id(frontend_span.context.trace_id)
        print(f"Trace ID: {trace_id}")
        print(f"View trace in Datadog: https://app.datadoghq.com/apm/traces?query=%40otel.trace_id%3A{trace_id}")

def handle_checkout():
    with tracer.start_as_current_span("checkout") as checkout_span:
        print("Handling checkout...")
        checkout_span.set_attribute("order_num", int(datetime.datetime.timestamp(datetime.datetime.now())*1000) % 100000)
        time.sleep(random.random())
        handle_payment()
        time.sleep(random.random())
        handle_shipping()
        time.sleep(random.random())
    
        checkout_span.set_status(Status(StatusCode.OK))
        print("Checkout complete.")
        
def handle_payment():
    with tracer.start_as_current_span("payment") as payment_span:
        print("Handling payment...")
        payment_span.set_attribute("payment_id", str(uuid.uuid4()))
        if (random.random() < 0.1):
            payment_span.set_status(Status(StatusCode.ERROR))
        else:
            time.sleep(random.random())
            payment_span.set_status(Status(StatusCode.OK))
        print("Payment complete.")
            
def handle_shipping():
    with tracer.start_as_current_span("shipping") as shipping_span:
        print("Handling shipping...")
        shipping_span.set_attribute("tracking_num", str(uuid.uuid4()))
        time.sleep(random.random())
        print("Shipping complete.")

frontend()    

<div class="alert alert-block alert-warning"><b>NOTE:</b> What's missing from this trace?</div>

<img src="imgs/flowmap-single.png" width="600"/>

## Refactor to make it multi-service

We've refactored the original example to get a new **TraceProvider** instance *with a unique service name* for each of our handlers. This simulates each span being emitted by a separate microservice.

In [ ]:
def getTracer(service_name):
    provider = TracerProvider(resource=Resource.create({
        "service.name": service_name,
        "service.instance.id": str(uuid.uuid4()),
        "deployment.environment": "otel",
        "host.name": socket.gethostname(),
        "datadog.host.use_as_metadata": True,
    }))
    provider.add_span_processor(BatchSpanProcessor(OTLPSpanExporter(endpoint="localhost:4317", insecure=True)))
    return trace.get_tracer("python", tracer_provider=provider)
    
def frontend():
    frontend_tracer = getTracer("frontend")
    with frontend_tracer.start_as_current_span("frontend") as frontend_span:
        print("Processing web transaction...")
        time.sleep(random.random())
        handle_checkout()
        time.sleep(random.random())
    
        frontend_span.set_status(Status(StatusCode.OK))
        print("Transaction complete.")
        trace_id = trace.format_trace_id(frontend_span.context.trace_id)
        print(f"Trace ID: {trace_id}")
        print(f"View trace in Datadog: https://app.datadoghq.com/apm/traces?query=%40otel.trace_id%3A{trace_id}")

def handle_checkout():
    checkout_tracer = getTracer("checkout")
    with checkout_tracer.start_as_current_span("checkout") as checkout_span:
        print("Handling checkout...")
        checkout_span.set_attribute("order_num", int(datetime.datetime.timestamp(datetime.datetime.now())*1000) % 100000)
        time.sleep(random.random())
        handle_payment()
        time.sleep(random.random())
        handle_shipping()
        time.sleep(random.random())
    
        checkout_span.set_status(Status(StatusCode.OK))
        print("Checkout complete.")
        
def handle_payment():
    payment_tracer = getTracer("payment")
    with payment_tracer.start_as_current_span("payment") as payment_span:
        print("Handling payment...")
        payment_span.set_attribute("payment_id", str(uuid.uuid4()))
        if (random.random() < 0.1):
            payment_span.set_status(Status(StatusCode.ERROR))
        else:
            time.sleep(random.random())
            payment_span.set_status(Status(StatusCode.OK))
        print("Payment complete.")
    
def handle_shipping():
    shipping_tracer = getTracer("shipping")
    with shipping_tracer.start_as_current_span("shipping") as shipping_span:
        print("Handling shipping...")
        shipping_span.set_attribute("tracking_num", str(uuid.uuid4()))
        time.sleep(random.random())
        print("Shipping complete.")

frontend()    

### Updated flowmap with each service represented separately
<img src="imgs/flowmap-multi.png" width="600"/>

#### End of Section